In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Fetch historical stock data
def get_stock_data(stock_symbol, start_date="2015-01-01"):
    stock = yf.download(stock_symbol, start=start_date)
    return stock[['Close']]  # Only keep closing prices

# Get data for a specific stock (e.g., "RELIANCE.NS" for Reliance Industries)
stock_symbol = "RELIANCE.NS"  # NSE India format
df = get_stock_data(stock_symbol)
df.head()



[*********************100%***********************]  1 of 1 completed


Price,Close
Ticker,RELIANCE.NS
Date,
2015-01-01,183.761795
2015-01-02,183.275467
2015-01-05,181.267914
2015-01-06,173.041138
2015-01-07,176.807877


In [26]:
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df)

# Convert data into sequences for LSTM
def create_sequences(data, time_step=60):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

time_step = 60  # Look back 60 days
X, Y = create_sequences(df_scaled, time_step)

# Reshape for LSTM (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split into training (80%) and testing (20%)
split = int(0.8 * len(X))
X_train, Y_train = X[:split], Y[:split]
X_test, Y_test = X[split:], Y[split:]


In [27]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=1)


d:\stx\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - loss: 0.0410
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0019
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - loss: 0.0017
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - loss: 0.0014
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0014
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - loss: 0.0013
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 0.0011
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 0.0011
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 9.2718e-04
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - loss: 0.0012


In [28]:
import plotly.graph_objects as go
import numpy as np

# Downsample data by selecting every n-th point
n = 5  # Adjust to control density (higher n = fewer points)
x_values = df.index[-len(Y_test):][::n]
actual_y_values = actual_prices.flatten()[::n]
predicted_y_values = predictions.flatten()[::n]

# Create interactive figure
fig = go.Figure()

# Add actual prices trace
fig.add_trace(go.Scatter(
    x=x_values, 
    y=actual_y_values, 
    mode='lines+markers', 
    name="Actual Price",
    hoverinfo="x+y+text",
    text=[f"Actual: {y:.2f}" for y in actual_y_values],  # Data labels in hover tooltip
    marker=dict(size=7)  # Adjust marker size
))

# Add predicted prices trace
fig.add_trace(go.Scatter(
    x=x_values, 
    y=predicted_y_values, 
    mode='lines+markers', 
    name="Predicted Price",
    hoverinfo="x+y+text",
    text=[f"Predicted: {y:.2f}" for y in predicted_y_values],  # Data labels in hover tooltip
    marker=dict(size=7, symbol="square")  # Different marker for distinction
))

# Customize layout
fig.update_layout(
    title=f"Stock Price Prediction for {stock_symbol}",
    xaxis_title="Date",
    yaxis_title="Stock Price",
    hovermode="x unified",  # Unifies hover info across traces
    template="plotly_white"  # Clean white background
)

fig.show()


In [29]:
# Get last 60 days' data
last_60_days = df_scaled[-time_step:]
last_60_days = last_60_days.reshape(1, time_step, 1)

# Predict next day price
predicted_price = model.predict(last_60_days)
predicted_price = scaler.inverse_transform(predicted_price)[0][0]

print(f"Predicted next day's price for {stock_symbol}: ₹{predicted_price:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step
Predicted next day's price for RELIANCE.NS: ₹1244.39


In [30]:
# Compute Relative Strength Index (RSI)
def compute_RSI(data, window=14):
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

df['RSI'] = compute_RSI(df)

In [31]:
# Compute Moving Averages (50-day & 200-day)
df['MA_50'] = df['Close'].rolling(window=50).mean()
df['MA_200'] = df['Close'].rolling(window=200).mean()

In [33]:
df

Price,Close,RSI,MA_50,MA_200
Ticker,RELIANCE.NS,,,
Date,,,,
2015-01-01,183.761795,NaN,NaN,NaN
2015-01-02,183.275467,NaN,NaN,NaN
2015-01-05,181.267914,NaN,NaN,NaN
2015-01-06,173.041138,NaN,NaN,NaN
2015-01-07,176.807877,NaN,NaN,NaN
...,...,...,...,...
2025-03-21,1276.349976,67.996215,1241.588999,1364.284269
2025-03-24,1302.099976,81.341293,1242.792998,1363.715048
